# **Women Cloth Reviews Prediction With Multi Nomial Naive Bayes**

The Multi Nomial Naive Bayes classifier is suitable for classification with discrete features.( eg. Word counts for text classification).The Multinomial distribution normally requires integer feature counts.However,in practice,fractional counts such as tf-idf may also work.


#Import Library


In [132]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [131]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import CSV as dataframe

In [133]:
df=pd.read_csv("/content/drive/MyDrive/Share Womens Clothing E-Commerce Reviews.csv")


# Get five rows of dataframe

In [134]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


# Get information of Dataframe

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


# Get shape of dataframe


In [136]:
df.shape

(23486, 11)

# Get summary of statistics

In [137]:
df.describe()

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,11742.500000,918.118709,43.198544,4.196032,0.822362,2.535936
std,6779.968547,203.298980,12.279544,1.110031,0.382216,5.702202
min,0.000000,0.000000,18.000000,1.000000,0.000000,0.000000
25%,5871.250000,861.000000,34.000000,4.000000,1.000000,0.000000
50%,11742.500000,936.000000,41.000000,5.000000,1.000000,1.000000
75%,17613.750000,1078.000000,52.000000,5.000000,1.000000,3.000000
max,23485.000000,1205.000000,99.000000,5.000000,1.000000,122.000000


# Missing Values

Remove missing values in Review Text columns with no review text

In [138]:
df.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [139]:
df[df['Review Text']==""]=np.NaN

In [140]:
df['Review Text'].fillna("No Review",inplace=True)

In [141]:
df.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                   0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [142]:
df['Review Text']

0        Absolutely wonderful - silky and sexy and comf...
1        Love this dress!  it's sooo pretty.  i happene...
2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 23486, dtype: object

# Get the columns names

In [143]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

#Define y (Dependent or label or target variable) and X (independent or features or attribute variable)


In [144]:
X=df['Review Text']

In [145]:
y=df['Rating']

In [146]:
df['Rating'].value_counts()

5.0    13131
4.0     5077
3.0     2871
2.0     1565
1.0      842
Name: Rating, dtype: int64

# Get Train Test split

In [147]:
from sklearn.model_selection import train_test_split

In [148]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y,random_state=2529)

In [149]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((16440,), (7046,), (16440,), (7046,))

# Get feature text conversion to tokens

In [150]:
from sklearn.feature_extraction.text import CountVectorizer

In [151]:
cv=CountVectorizer(lowercase=True,analyzer='word',ngram_range=(2,3),stop_words='english',max_features=5000)

In [152]:
X_train=cv.fit_transform(X_train)

In [153]:
cv.get_feature_names_out()

array(['10 12', '10 bought', '10 fit', ..., 'yellow color', 'yoga pants',
       'zipper little'], dtype=object)

In [154]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [155]:
X_test=cv.fit_transform(X_test)

In [156]:
cv.get_feature_names_out()

array(['10 12', '10 dress', '10 fit', ..., 'years come', 'years old',
       'yoga pants'], dtype=object)

In [157]:
X_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Get Model Train

In [158]:
from sklearn.naive_bayes import MultinomialNB

In [159]:
model=MultinomialNB()

In [160]:
model.fit(X_train, y_train)

MultinomialNB()

# Get Model Prediction

In [161]:
y_pred=model.predict(X_test)

In [162]:
y_pred.shape

(7046,)

In [163]:
y_pred

array([1., 5., 5., ..., 5., 5., 5.])

# Get probability of each predicted class

In [164]:
model.predict_proba(X_test)

array([[0.71118473, 0.02625165, 0.15465118, 0.01496876, 0.09294369],
       [0.02416867, 0.04769471, 0.35268622, 0.16185007, 0.41360034],
       [0.03582725, 0.06660584, 0.12226277, 0.21618005, 0.55912409],
       ...,
       [0.02320281, 0.08950939, 0.08962183, 0.16719203, 0.63047394],
       [0.01167675, 0.00202714, 0.08539004, 0.34347398, 0.55743209],
       [0.03959824, 0.05612822, 0.00688869, 0.1560574 , 0.74132745]])

# Get model evaluation

In [165]:
from sklearn.metrics import confusion_matrix,classification_report

In [166]:
print(confusion_matrix(y_test,y_pred))

[[  15   13   45   36  144]
 [  43   43   86   85  213]
 [ 116   78  113  166  388]
 [ 166  108  194  336  719]
 [ 371  272  349  722 2225]]


In [167]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         1.0       0.02      0.06      0.03       253
         2.0       0.08      0.09      0.09       470
         3.0       0.14      0.13      0.14       861
         4.0       0.25      0.22      0.23      1523
         5.0       0.60      0.56      0.58      3939

    accuracy                           0.39      7046
   macro avg       0.22      0.21      0.21      7046
weighted avg       0.42      0.39      0.40      7046



# Recategories ratings as poor(0) and good(1)

In [168]:
df['Rating'].value_counts()

5.0    13131
4.0     5077
3.0     2871
2.0     1565
1.0      842
Name: Rating, dtype: int64

Re-Rating as 1,2,3 and 4,5 as 1

In [169]:
df.replace({'Rating':{1:0,2:0,3:0,4:1,5:1}},inplace=True)

In [170]:
y=df['Rating']

In [171]:
X=df['Review Text']

#Train Test split

In [172]:
from sklearn.model_selection import train_test_split

In [173]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y,random_state=2529)

In [174]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((16440,), (7046,), (16440,), (7046,))

# Get feature text conversion to tokens

In [175]:
from sklearn.feature_extraction.text import CountVectorizer

In [176]:
cv=CountVectorizer(lowercase=True,analyzer='word',ngram_range=(2,3),stop_words='english',max_features=5000)

In [177]:

X_train=cv.fit_transform(X_train)

In [178]:
X_test=cv.fit_transform(X_test)

# Get Model Re-train

In [179]:
from sklearn.naive_bayes import MultinomialNB

In [180]:
model=MultinomialNB()

In [181]:
model.fit(X_train,y_train)

MultinomialNB()

# Get model prediction

In [182]:
y_pred=model.predict(X_test)

In [183]:
y_pred.shape

(7046,)

In [184]:
y_pred

array([1., 1., 1., ..., 1., 1., 1.])

# Get model evaluation

In [185]:
from sklearn.metrics import confusion_matrix,classification_report

In [186]:
print(confusion_matrix(y_test,y_pred))

[[ 449 1134]
 [ 989 4474]]


In [187]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.31      0.28      0.30      1583
         1.0       0.80      0.82      0.81      5463

    accuracy                           0.70      7046
   macro avg       0.56      0.55      0.55      7046
weighted avg       0.69      0.70      0.69      7046

